# Block Aligner Benchmark Analysis and Visualizations

This notebook contains code for collecting, cleaning, and analyzing data produced by block aligner's experiments.

To run this, you will need to install all the libraries imported below, along with [altair-saver](https://github.com/altair-viz/altair_saver) and [altair-data-server](https://github.com/altair-viz/altair_data_server), which has some extra dependencies for PDF saving.

Run each cell one by one to reproduce the experiments. This may take a while. For accurate benchmarking, it is recommended to run the entire notebook in the command line with `nbconvert`.

In [1]:
import altair as alt
from altair_saver import save
from altair import datum
import pandas as pd
from io import StringIO

alt.data_transformers.enable("data_server")

DataTransformerRegistry.enable('data_server')

In [2]:
def csv_to_pandas(csv, d = "\\s*,\\s*", t = None):
    s = StringIO("\n".join(csv))
    data = pd.read_csv(s, sep = d, thousands = t, comment = "#", engine = "python")
    return data

## Prefix Scan Benchmark

In [3]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- prefix_scan | grep 'bench:' | awk '{print $2"\t"$5}'
output.insert(0, "algorithm\ttime")
output

['algorithm\ttime', 'bench_naive_prefix_scan\t33', 'bench_opt_prefix_scan\t17']

In [4]:
data = csv_to_pandas(output, d = "\t", t = ",")
data

,algorithm,time
0,bench_naive_prefix_scan,33
1,bench_opt_prefix_scan,17


In [5]:
data["algorithm"] = data["algorithm"].map({
    "bench_naive_prefix_scan": "naive",
    "bench_opt_prefix_scan": "ours"
})
data

,algorithm,time
0,naive,33
1,ours,17


Prefix Scan Benchmark (AVX2)

In [6]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("time", axis = alt.Axis(title = "time (ns)")),
    y = "algorithm",
    color = alt.Color("algorithm", legend = None)
).properties(
    width = 150
)
save(c, "prefix_scan_bench.pdf")
c

alt.Chart(...)

## Random Data Benchmark

In [7]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- bench_ | grep 'bench:' | grep -v 'prefix_scan' | awk '{print $2"\t"$5}'
output

['bench_parasailors_aa_1000_10000\t44,982,764',
 'bench_parasailors_aa_100_1000\t476,651',
 'bench_parasailors_aa_10_100\t17,258',
 'bench_rustbio_aa_100_1000\t15,359,585',
 'bench_rustbio_aa_10_100\t160,203',
 'bench_scan_aa_1000_10000\t234,717',
 'bench_scan_aa_1000_10000_insert\t8,685,719',
 'bench_scan_aa_1000_10000_small\t220,743',
 'bench_scan_aa_1000_10000_trace\t1,850,610',
 'bench_scan_aa_100_1000\t24,079',
 'bench_scan_aa_100_1000_insert\t52,457',
 'bench_scan_aa_100_1000_small\t22,769',
 'bench_scan_aa_100_1000_trace\t507,608',
 'bench_scan_aa_10_100\t3,823',
 'bench_scan_aa_10_100_insert\t3,919',
 'bench_scan_aa_10_100_small\t3,428',
 'bench_scan_aa_10_100_trace\t361,683',
 'bench_scan_nuc_1000_10000\t213,811',
 'bench_scan_nuc_100_1000\t23,643',
 'bench_triple_accel_1000_10000\t7,512,349',
 'bench_triple_accel_100_1000\t24,088']

In [8]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["parasailors_aa", "rustbio_aa", "scan_aa", "scan_nuc", "triple_accel"]
new_names = ["parasailors\tprotein", "rust bio\tprotein", "ours\tprotein", "ours\tnucleotide", "triple accel\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'parasailors\tprotein\t1000\t10000\tdefault\t44,982,764',
 'parasailors\tprotein\t100\t1000\tdefault\t476,651',
 'parasailors\tprotein\t10\t100\tdefault\t17,258',
 'rust bio\tprotein\t100\t1000\tdefault\t15,359,585',
 'rust bio\tprotein\t10\t100\tdefault\t160,203',
 'ours\tprotein\t1000\t10000\tdefault\t234,717',
 'ours\tprotein\t1000\t10000\tinsert\t8,685,719',
 'ours\tprotein\t1000\t10000\tsmall\t220,743',
 'ours\tprotein\t1000\t10000\ttrace\t1,850,610',
 'ours\tprotein\t100\t1000\tdefault\t24,079',
 'ours\tprotein\t100\t1000\tinsert\t52,457',
 'ours\tprotein\t100\t1000\tsmall\t22,769',
 'ours\tprotein\t100\t1000\ttrace\t507,608',
 'ours\tprotein\t10\t100\tdefault\t3,823',
 'ours\tprotein\t10\t100\tinsert\t3,919',
 'ours\tprotein\t10\t100\tsmall\t3,428',
 'ours\tprotein\t10\t100\ttrace\t361,683',
 'ours\tnucleotide\t1000\t10000\tdefault\t213,811',
 'ours\tnucleotide\t100\t1000\tdefault\t23,643',
 'triple accel\tnucleotide\t1000\t100

In [9]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,parasailors,protein,1000,10000,default,44982764
1,parasailors,protein,100,1000,default,476651
2,parasailors,protein,10,100,default,17258
3,rust bio,protein,100,1000,default,15359585
4,rust bio,protein,10,100,default,160203
5,ours,protein,1000,10000,default,234717
6,ours,protein,1000,10000,insert,8685719
7,ours,protein,1000,10000,small,220743
8,ours,protein,1000,10000,trace,1850610
9,ours,protein,100,1000,default,24079


In [10]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (AVX2)

In [11]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench.pdf")
c

alt.Chart(...)

Random DNA Sequences Benchmark (AVX2)

In [12]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000])),
    shape = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000]))
).transform_filter(
    datum.alphabet == "nucleotide"
).properties(
    width = 200,
    height = 50
)
save(c, "random_dna_bench.pdf")
c

alt.Chart(...)

## Uniclust 30 Data Benchmark

In [13]:
output = !cd .. && cargo run --example uc_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, size, time',
 'ours (no trace), uc30, 32-32, 0.055945224',
 'ours (no trace), uc30 0.95, 32-32, 0.06045113',
 'ours (no trace), uc30, 32-256, 0.088573443',
 'ours (no trace), uc30 0.95, 32-256, 0.077502856',
 'ours (no trace), uc30, 256-256, 0.201014255',
 'ours (no trace), uc30 0.95, 256-256, 0.220710363',
 'ours (trace), uc30, 32-256, 0.18067667',
 'ours (trace), uc30 0.95, 32-256, 0.159931929',
 'parasail, uc30, full, 0.788206077',
 'parasail, uc30 0.95, full, 0.915880307']

In [14]:
data = csv_to_pandas(output)
data

,algorithm,dataset,size,time
0,ours (no trace),uc30,32-32,0.055945
1,ours (no trace),uc30 0.95,32-32,0.060451
2,ours (no trace),uc30,32-256,0.088573
3,ours (no trace),uc30 0.95,32-256,0.077503
4,ours (no trace),uc30,256-256,0.201014
5,ours (no trace),uc30 0.95,256-256,0.220710
6,ours (trace),uc30,32-256,0.180677
7,ours (trace),uc30 0.95,32-256,0.159932
8,parasail,uc30,full,0.788206
9,parasail,uc30 0.95,full,0.915880


Uniclust30 Benchmark (AVX2)

In [15]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", axis = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
    color = "algorithm"
).transform_filter(
    (datum.size == "32-256") | (datum.algorithm == "parasail")
).properties(
    width = 50,
    height = 100
).configure_range(
    category = {"scheme": "dark2"}
)
save(c, "uniclust30_bench.pdf")
c

alt.Chart(...)

Uniclust30 Block Size Benchmark (AVX2)

In [16]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", axis = None, sort = ["32-32", "32-256", "256-256"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
    color = alt.Color("size", sort = ["32-32", "32-256", "256-256"])
).transform_filter(
    datum.algorithm == "ours (no trace)"
).properties(
    width = 50,
    height = 100
)
save(c, "uniclust30_size_bench.pdf")
c

alt.Chart(...)

## Nanopore Data Benchmark Setup

To run the benchmarks below, you need to clone the following repos, place them in the same directory where this repo (block aligner) is located, and follow their setup instructions:
* [diff-bench-paper](https://github.com/Daniel-Liu-c0deb0t/diff-bench-paper)
* [adaptivebandbench](https://github.com/Daniel-Liu-c0deb0t/adaptivebandbench)

## Nanopore Data Benchmark

In [17]:
output = !cd .. && cargo run --example nanopore_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, time',
 'ours (no trace 32-32), nanopore 25kbp, 0.963649697',
 'ours (no trace 32-32), random, 2.376020548',
 'ours (trace 32-32), nanopore 25kbp, 1.315417409',
 'ours (trace 32-32), random, 3.294219892',
 'ours (trace 32-64), nanopore 25kbp, 1.6175370980000001',
 'ours (trace 32-64), random, 3.418951881']

In [18]:
data = csv_to_pandas(output)
data

,algorithm,dataset,time
0,ours (no trace 32-32),nanopore 25kbp,0.963650
1,ours (no trace 32-32),random,2.376021
2,ours (trace 32-32),nanopore 25kbp,1.315417
3,ours (trace 32-32),random,3.294220
4,ours (trace 32-64),nanopore 25kbp,1.617537
5,ours (trace 32-64),random,3.418952


In [19]:
output2 = !cd ../../diff-bench-paper/supplementary_data/benchmark_codes && ./custom_bench.sh

for i, o in enumerate(output2):
    if o.startswith("cells("):
        break
output2 = output2[i + 1:]

output2.insert(0, "algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail")
output2

['algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail',
 'editdist\t467660000\t167184000\t65830000\t700674000\t6880489\t0',
 'non-diff\t688273000\t261696000\t59112000\t1009081000\t27124786\t52',
 'diff-raw\t609221000\t205256000\t62630000\t877107000\t27291141\t32',
 'libgaba\t434103000\t150617000\t31617000\t616337000\t27121546\t53',
 'edlib\t28126247000\t19417377000\t106316000\t47649940000\t37\t0',
 'seqan\t90358733000\t0\t0\t90358733000\t0\t0']

In [20]:
data2 = csv_to_pandas(output2, d = "\t")
data2

,algorithm,fill time,trace time,convert time,total time,score,fail
0,editdist,467660000,167184000,65830000,700674000,6880489,0
1,non-diff,688273000,261696000,59112000,1009081000,27124786,52
2,diff-raw,609221000,205256000,62630000,877107000,27291141,32
3,libgaba,434103000,150617000,31617000,616337000,27121546,53
4,edlib,28126247000,19417377000,106316000,47649940000,37,0
5,seqan,90358733000,0,0,90358733000,0,0


In [21]:
cleaned2 = data2.drop(columns = ["trace time", "convert time", "total time", "score", "fail"])
cleaned2 = cleaned2.rename(columns = {"fill time": "time"})
cleaned2["time"] /= 1e9
cleaned2

,algorithm,time
0,editdist,0.467660
1,non-diff,0.688273
2,diff-raw,0.609221
3,libgaba,0.434103
4,edlib,28.126247
5,seqan,90.358733


In [22]:
cleaned = data.drop(index = [1, 3, 5])
cleaned = cleaned.drop(columns = ["dataset"])
cleaned = cleaned.append(cleaned2, ignore_index = True)
cleaned

,algorithm,time
0,ours (no trace 32-32),0.963650
1,ours (trace 32-32),1.315417
2,ours (trace 32-64),1.617537
3,editdist,0.467660
4,non-diff,0.688273
5,diff-raw,0.609221
6,libgaba,0.434103
7,edlib,28.126247
8,seqan,90.358733


25kbp Nanopore Reads Benchmark (AVX2)

In [23]:
chart1 = alt.Chart(cleaned).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm != "ours (trace 32-32)") & (datum.algorithm != "ours (no trace 32-32)") & (datum.algorithm != "ours (trace 32-64)"))

chart2 = alt.Chart(cleaned).mark_point(color = "red", filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm == "ours (trace 32-32)") | (datum.algorithm == "ours (no trace 32-32)") | (datum.algorithm == "ours (trace 32-64)"))

c = (chart1 + chart2).properties(
    width = 150,
    height = 150
)
save(c, "nanopore_bench.pdf")
c

alt.LayerChart(...)

## Sequence-to-Profile Alignment Benchmark

In [24]:
output = !cd .. && cargo run --example pssm_bench --release --features simd_avx2 --quiet
output

['size, time',
 '32-32, 0.153963937',
 '32-64, 0.178911012',
 '32-128, 0.211135543',
 '2048-2048, 4.163908914',
 '# Done!']

In [25]:
data = csv_to_pandas(output)
data

,size,time
0,32-32,0.153964
1,32-64,0.178911
2,32-128,0.211136
3,2048-2048,4.163909


SCOP Sequence-to-Profile Alignment Benchmark (AVX2)

In [26]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", sort = ["32-32", "32-64", "32-128", "2048-2048"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("size", sort = ["32-32", "32-64", "32-128", "2048-2048"], legend = None)
).properties(
    width = 60,
    height = 100
)
save(c, "pssm_size_bench.pdf")
c

alt.Chart(...)

## WASM SIMD

[Wasmtime](https://wasmtime.dev/) is needed to run the webassembly code.

In [27]:
output = !CARGO_TARGET_WASM32_WASI_RUNNER="wasmtime --wasm-features simd --" cargo bench --target=wasm32-wasi --features simd_wasm --quiet -- --nocapture | grep 'bench:' | awk '{print $2"\t"$5}'
output

['bench_rustbio_aa_100_1000\t23,381,030',
 'bench_rustbio_aa_10_100\t242,949',
 'bench_scan_aa_1000_10000\t999,211',
 'bench_scan_aa_1000_10000_insert\t11,895,694',
 'bench_scan_aa_1000_10000_small\t660,273',
 'bench_scan_aa_1000_10000_trace\t2,713,454',
 'bench_scan_aa_100_1000\t90,206',
 'bench_scan_aa_100_1000_insert\t173,977',
 'bench_scan_aa_100_1000_small\t63,632',
 'bench_scan_aa_100_1000_trace\t395,953',
 'bench_scan_aa_10_100\t8,109',
 'bench_scan_aa_10_100_insert\t12,253',
 'bench_scan_aa_10_100_small\t5,849',
 'bench_scan_aa_10_100_trace\t192,409',
 'bench_scan_nuc_1000_10000\t667,550',
 'bench_scan_nuc_100_1000\t65,636']

In [28]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["rustbio_aa", "scan_aa", "scan_nuc"]
new_names = ["rust bio\tprotein", "ours\tprotein", "ours\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'rust bio\tprotein\t100\t1000\tdefault\t23,381,030',
 'rust bio\tprotein\t10\t100\tdefault\t242,949',
 'ours\tprotein\t1000\t10000\tdefault\t999,211',
 'ours\tprotein\t1000\t10000\tinsert\t11,895,694',
 'ours\tprotein\t1000\t10000\tsmall\t660,273',
 'ours\tprotein\t1000\t10000\ttrace\t2,713,454',
 'ours\tprotein\t100\t1000\tdefault\t90,206',
 'ours\tprotein\t100\t1000\tinsert\t173,977',
 'ours\tprotein\t100\t1000\tsmall\t63,632',
 'ours\tprotein\t100\t1000\ttrace\t395,953',
 'ours\tprotein\t10\t100\tdefault\t8,109',
 'ours\tprotein\t10\t100\tinsert\t12,253',
 'ours\tprotein\t10\t100\tsmall\t5,849',
 'ours\tprotein\t10\t100\ttrace\t192,409',
 'ours\tnucleotide\t1000\t10000\tdefault\t667,550',
 'ours\tnucleotide\t100\t1000\tdefault\t65,636']

In [29]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,rust bio,protein,100,1000,default,23381030
1,rust bio,protein,10,100,default,242949
2,ours,protein,1000,10000,default,999211
3,ours,protein,1000,10000,insert,11895694
4,ours,protein,1000,10000,small,660273
5,ours,protein,1000,10000,trace,2713454
6,ours,protein,100,1000,default,90206
7,ours,protein,100,1000,insert,173977
8,ours,protein,100,1000,small,63632
9,ours,protein,100,1000,trace,395953


In [30]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (WASM SIMD)

In [31]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench_wasm.pdf")
c

alt.Chart(...)